In [1]:
!pip install torch
!pip install torchvision

In [2]:
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from mls import mls
import json

In [3]:
train_dataset = mls.CustomDataSet("./dataset/processed/train_data.csv")
val_dataset = mls.CustomDataSet("./dataset/processed/test_data.csv")

In [4]:
batch_size = 16
shuffle_dataset = True
num_epochs = 150
in_features = len(train_dataset[0][0])
out_classes = 5
save_model = True
save_interval = 10
model_p = './checkpoints/'

In [5]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=True)

In [6]:
model = mls.MVC(in_features).double()
learning_rate = 0.1
loss_f = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min')

In [ ]:
train_loss = []
train_acc = []
val_loss = []
val_acc = []
lrs = []
best_loss = np.inf
for epoch in range(num_epochs):
    t_loss, t_acc = mls.training_epoch(model, optimizer, loss_f, train_loader)
    t_loss = t_loss/len(train_dataset)
    t_acc = 100*(t_acc/len(train_dataset))
    
    v_loss, v_acc = mls.validation_epoch(model, loss_f, validation_loader)
    v_loss = v_loss/len(val_dataset)
    v_acc = 100*(v_acc/len(val_dataset))
    
    train_loss.append(t_loss)
    val_loss.append(v_loss)
    train_acc.append(t_acc)
    val_acc.append(v_loss)
    lrs.append(optimizer.param_groups[0]['lr'])
    scheduler.step(v_loss)
    
    if epoch%save_interval==0 and save_model:
        isBest = v_loss<best_loss
        mls.save_model(model_p, epoch, model, optimizer, v_loss, isBest) 
        
    print(f'e{epoch}: Train Loss: {t_loss:.04f}; Val Loss: {v_loss:.04f} acc: {v_acc:.04f}')
        

e0: Train Loss: 0.5024; Val Loss: 0.2443 acc: 91.5333
e1: Train Loss: 0.2207; Val Loss: 0.1993 acc: 93.1333
e2: Train Loss: 0.1902; Val Loss: 0.1758 acc: 93.9200
e3: Train Loss: 0.1760; Val Loss: 0.1653 acc: 93.9867
e4: Train Loss: 0.1612; Val Loss: 0.1945 acc: 92.6133
e5: Train Loss: 0.1505; Val Loss: 0.2068 acc: 92.2400
e6: Train Loss: 0.1382; Val Loss: 0.1615 acc: 93.7867
e7: Train Loss: 0.1341; Val Loss: 0.2029 acc: 92.3067
e8: Train Loss: 0.1276; Val Loss: 0.1349 acc: 95.1333
e9: Train Loss: 0.1164; Val Loss: 0.1750 acc: 93.9600
e10: Train Loss: 0.1106; Val Loss: 0.1378 acc: 95.0533
e11: Train Loss: 0.1035; Val Loss: 0.1269 acc: 95.4000
e12: Train Loss: 0.0975; Val Loss: 0.1740 acc: 93.9733
e13: Train Loss: 0.0901; Val Loss: 0.1379 acc: 94.9867
e14: Train Loss: 0.0848; Val Loss: 0.1385 acc: 95.1467
e15: Train Loss: 0.0806; Val Loss: 0.1256 acc: 95.7333
e16: Train Loss: 0.0742; Val Loss: 0.1357 acc: 95.2533
e17: Train Loss: 0.0710; Val Loss: 0.1127 acc: 96.2000
e18: Train Loss: 0.0

In [ ]:
with open(f'{model_p}/log.json', 'w') as fout:
    training_info ={
        'train_loss':train_loss,
        'train_acc':train_acc,
        'val_loss':val_loss,
        'val_acc':val_acc,
        'lrs':lrs,
    }
    json_dumps_str = json.dumps(training_info, indent=4)
    print(json_dumps_str, file=fout)

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(3, 1, figsize=(20,5))
axs[0].plot(train_loss, color='blue')
axs[0].plot(val_loss, color='black')

axs[1].plot(train_acc, color='blue')
axs[1].plot(val_acc, color='black')

axs[2].plot(lrs, color='black')